In [9]:
import lore

from prepare_dataset import *
from neighbor_generator import *

from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [10]:
path_data = '../../dataset/heart-disease/'

dataset_name = 'heart-disease.csv'
dataset = prepare_disease_dataset(dataset_name, path_data)

X, y = dataset['X'], dataset['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.preprocessing import StandardScaler
transform = StandardScaler()
X_train[:, :6] = transform.fit_transform(X_train[:, :6])
X_test[:, :6] = transform.transform(X_test[:, :6])

noise_std = 0.2
np.random.seed(0)
X_test_noisy = X_test + noise_std * np.random.randn(*X_test.shape)

import joblib
blackbox = joblib.load('../../saved_model/heart-disease/MLP.pkl')

X2E = X_test_noisy
y2E = blackbox.predict(X2E)
y2E = np.asarray([dataset['possible_outcomes'][i] for i in y2E])

In [11]:
explanations = [] 

# First loop over idx_record2explain, generate explanation, and store the results in explanations
for idx_record2explain in range(90):
    explanation, infos = lore.explain(idx_record2explain, X2E, dataset, blackbox,
                                      ng_function=genetic_neighborhood,
                                      discrete_use_probabilities=True,
                                      continuous_function_estimation=False,
                                      returns_infos=True,
                                      path=path_data, sep=';', log=False)
    explanations.append((explanation, infos))

In [12]:
# Then loop over k, and for each k iterate through all explanations
for k in range(1, 26):
    test_data = pd.read_csv('../../dataset/heart-disease/test_data.csv')
    bound = pd.read_csv('../../dataset/heart-disease/bound.csv')
    for idx_record2explain in range(90):
        explanation, infos = explanations[idx_record2explain] # Use the previously stored explanation
        dfX2E = build_df2explain(blackbox, X2E, dataset).to_dict('records')
        dfx = dfX2E[idx_record2explain]
        # x = build_df2explain(blackbox, X2E[idx_record2explain].reshape(1, -1), dataset).to_dict('records')[0]
        keys_view = explanation[0][1].keys()
        keys_list = list(keys_view)
        length = len(keys_list)
        if length > k:
            for j in range(0,k):
                test_data.at[idx_record2explain,keys_list[j]]=bound.at[1,keys_list[j]]
        elif length <= k:
            for j in range(0,length):
                test_data.at[idx_record2explain,keys_list[j]]=bound.at[1,keys_list[j]]
    filename = '../../explain_set/heart-disease/MLP/lore/noise/lore_explain_'+str(k)+'.csv'
    test_data.to_csv(filename,index=False)